# Exploring vector spaces

The first tutorial showed some very basic functionality about vectors and spaces containing them. This tutorial explores the topic a bit further and introduces some more interesting and useful functionality. As usual, we start by importing the `odl` module, and this time we also add `numpy` for later.

In [1]:
from __future__ import print_function  # For Python2 compatibility
import numpy as np
import odl

As seen before, one way to create a space of 3-element vectors with a certain data type is to use the `odl.fn` constructor:

In [2]:
odl.fn(3, 'int')

fn(3, 'int')

In [3]:
odl.fn(3, 'float')

rn(3)

In [4]:
odl.fn(3, 'complex')

cn(3)

The spaces with real or complex floating point data types have own constructors `rn` and `cn`, mostly because they are most heavily used. Mathematically, they stand for $n$-dimensional Euclidean spaces $\mathbb{R}^n$ or $\mathbb{C}^n$, respectively, where in our case, $n = 3$. In general, we usually write $\mathbb{F}$ for "$\mathbb{R}$ or $\mathbb{C}$" and $\mathbb{F}^n$ for the $n$-dimensional Euclidean vector space over the *field* $\mathbb{F}$.

Besides the arithmetic operations in Tutorial 1, Euclidean spaces have further structure, for example
- an [inner product](https://en.wikipedia.org/wiki/Inner_product_space) that allows measuring angles between vectors,
- a <a href="https://en.wikipedia.org/wiki/Norm_(mathematics)">norm</a> for measuring the length of a vector, and
- a <a href="https://en.wikipedia.org/wiki/Metric_(mathematics)">metric</a> to determine distances between points in space.

For simplicity, we look at the space $\mathbb{R}^n$ -- everything can be generalized for the complex case.

## Inner products

An inner product takes two vectors in the space $\mathbb{R}^n$ and produces a real number:

$$
    \langle\cdot, \cdot\rangle : \mathbb{R}^n \times \mathbb{R}^n \to \mathbb{R}.
$$

It is *linear in the first argument*, *conjuagte-symmetric* and *positive definite*. Check [here](https://en.wikipedia.org/wiki/Inner_product_space#Definition) for the exact definitions. The standard inner product on $\mathbb{R}^n$ is also known as "dot product" and defined as

$$
    \langle x, y\rangle_{\mathbb{R}^n} := \sum_{k=1}^n x_k y_k.
$$

In ODL, it is available as `inner` method both on the space, i.e., `space.inner(vec1, vec2)`, or on the space element, `vec1.inner(vec2)`. Let's try it out:

In [5]:
space = odl.rn(3)
vec1 = space.element([1, 2, 3])
vec2 = space.element([1, -1, 1])

In [6]:
space.inner(vec1, vec2)  # 1*1 + 2*(-1) + 3*1 = 2

2.0

In [7]:
vec1.inner(vec2)

2.0

We can calculate angles by the formula

$$
    \cos \angle(x, y) = \frac{\langle x, y \rangle}{\sqrt{\langle x, x \rangle \langle y, y \rangle}}
$$

For example, the angle between $x = (1, 1, 0)$ and $(1, 0, 0)$ should be 45 degrees:

In [8]:
RAD2DEG = 180 / np.pi  # conversion factor to degrees
x = space.element([1, 1, 0])
y = space.element([1, 0, 0])

cos_angle_rad = x.inner(y) / np.sqrt(x.inner(x) * y.inner(y))
angle_rad = np.arccos(cos_angle_rad)
angle_deg = angle_rad * RAD2DEG
print('The angle between {} and {} is {} degrees.'.format(x, y, angle_deg))

The angle between [1.0, 1.0, 0.0] and [1.0, 0.0, 0.0] is 45.00000000000001 degrees.


## Norms

A norm takes a vector in $\mathbb{R}^n$ and maps it to a postive number:

$$
    \|\cdot\| : \mathbb{R}^n \to [0, \infty)
$$

It is *positively one-homogeneous*, *positive definite* and satisfies the *triangle inequality*, see <a href="https://en.wikipedia.org/wiki/Norm_(mathematics)">here</a> for details. The standard norm on $\mathbb{R}^n$ is the root of the sum of squares of all components:

$$
    \|x\|_2 = \sqrt{\sum_{k=1}^n x_k^2}
$$

This is just the same as $\sqrt{\langle x, x\rangle}$ -- the norm is *induced* by the inner product.

Again, this function is available as a space method `space.norm(vec)` or as method on the element itself, `vec.norm()`:

In [9]:
x = space.element([2, 3, 6])  # has norm sqrt(2*2 + 3*3 + 6*6) = 7

In [10]:
space.norm(x)

7.0

In [11]:
x.norm()

7.0

There is a variety of norms on $\mathbb{R}^n$ that are not induced by an inner product. They are parametrized by a real number $p \in [1, \infty]$ and are defined as

$$
    \|x\|_p = \left( \sum_{k=1}^n |x_k|^p \right)^{1/p}\quad \text{for } p < \infty \text{ and}\quad \|x\|_\infty = \max_k |x_k|\quad \text{for } p = \infty.
$$

(In principle, also $p < 1$ is allowed, but the resulting function is no longer a norm)
In NumPy, norms are implemented in the function [`numpy.linalg.norm`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.norm.html), and the $p$ parameter can be chosen via the `ord` argument.

To use these norm variants in ODL, the space must be initialized with the `exponent` argument:

In [12]:
space = odl.rn(3, exponent=1)
space

rn(3, exponent=1.0)

Since the value of the exponent deviates from the default 2, it is also printed in the representation. Now we can go back to the example above and compute the norm in this space:

In [13]:
x = space.element([2, 3, 6])  # has norm 2 + 3 + 6 = 11
x.norm()

11.0

**Important:**
- Two otherwise equal spaces with differing exponent are considered different in ODL. The main reason for this is that only for `exponent=2.0`, the norm is induced by an inner product. For all other choices, `space.inner` is not defined!
- A vector "knows" to which space it belongs, i.e. if `x` was created in a space with one exponent, it is not considered and element of another space with a different exponent.

In [14]:
odl.rn(3, exponent=1) == odl.rn(3, exponent=2)

False

In [15]:
odl.rn(3, exponent=1).element([1, 2, 3]) in odl.rn(3)

False

If we try to call the `inner` method on either a space or an element of it, we get an error if `exponent != 2`:

In [16]:
space = odl.rn(3, exponent=1)
x = space.element([1, 2, 3])
y = space.element([1, 0, -1])

In [17]:
# We don't want to see a long traceback, only the error message.
# Don't worry too much about the details in the print statement :-).
try:
    space.inner(x, y)
except Exception as exc:
    print('{}: {}'.format(exc.__class__.__name__, exc))

NotImplementedError: no inner product defined for exponent != 2 (got 1.0)


In [18]:
try:
    x.inner(y)
except Exception as exc:
    print('{}: {}'.format(exc.__class__.__name__, exc))

NotImplementedError: no inner product defined for exponent != 2 (got 1.0)


## Metrics

A metric is a measure of distance between points in space, the "endpoints" of vectors. It takes two vectors and produces a positive real number:

$$
    d : \mathbb{R}^n \times \mathbb{R}^n \to [0, \infty)
$$

A metric is *symmetric*, *sub-additive* and *positive definite* -- check the details <a href="https://en.wikipedia.org/wiki/Metric_(mathematics)">here</a>. Most of the time, metrics are induced by norms in the form

$$
    d(x, y) = \|x - y\|.
$$

On a simple space like $\mathbb{R}^n$, metrics that are not of this form are unusual, but exist. We go with the default case here.

Using again the norm with exponent $p = 1$, we can compute distances with `space.dist(vec1, vec2)` or `vec1.dist(vec2)`:

In [19]:
space = odl.rn(3, exponent=1)
x = space.element([1, 2, 3])
y = space.element([1, 0, -1])
# Distance is: (1-1) + (2-0) + (3-(-1)) = 6
space.dist(x, y)

6.0

In [20]:
zero = space.element([0, 0, 0])
x.dist(zero) == x.norm()

True

## Some random but relevant details

- Calling `space.element()` without arguments allocates memory but does not initialize. It is similar to `numpy.empty`:

In [21]:
odl.rn(3).element()  # No guarantee for values

rn(3).element([1.0, 2.0, 3.0])

- Elements with all zeros or all ones can be created with `space.zero()` or `space.one()`, respectively:

In [22]:
odl.rn(3).zero()

rn(3).element([0.0, 0.0, 0.0])

In [23]:
odl.rn(3).one()

rn(3).element([1.0, 1.0, 1.0])

- The `rn` and `cn` also accept a `dtype` argument. This parameter also makes for distinct spaces:

In [24]:
space_f32 = odl.rn(3, dtype='float32')
space_f32

rn(3, 'float32')

In [25]:
space_f64 = odl.rn(3, dtype='float64')
space_f32 == space_f64

False

In [26]:
space_f32.element() in space_f64

False

- The data container of a space element (by default a Numpy array) can be retrieved by the `vec.asarray()` method. Note, however, that *mutating this array is not guaranteed to mutate the vector*. To be sure, the result must be re-assigned to the vector.

In [27]:
x = odl.rn(3).one()
x_arr = x.asarray()
x_arr[:] = 2
x[:] = x_arr  # no-op in this case
x

rn(3).element([2.0, 2.0, 2.0])

- NumPy arrays are merely wrapped if possible, i.e., if the default `'numpy'` back-end is used and if the data types of array and space are the same:

In [28]:
array_f32 = np.array([4, 5, 6], dtype='float32')
space_f32 = odl.rn(3, dtype='float32')
x_f32 = space_f32.element(array_f32)
x_f32[:] = -1
array_f32  # modified

array([-1., -1., -1.], dtype=float32)

In [29]:
space_f64 = odl.rn(3, dtype='float64')
x_f64 = space_f64.element(array_f32)
x_f64[:] = 17
array_f32  # same as before

array([-1., -1., -1.], dtype=float32)